                                  Practical:- Design a Domain Specific Language Using Lambda Calculus

**1)Define Lambda Calculus Primitives.**

In [10]:
TRUE  = lambda x: lambda y: x  # λx.λy.x
FALSE = lambda x: lambda y: y  # λx.λy.y
IF    = lambda p: lambda a: lambda b: p(a)(b)

**2)Define Church Numerals.**

In [16]:
ZERO  = lambda f: lambda x: x
ONE   = lambda f: lambda x: f(x)
SUCC  = lambda n: lambda f: lambda x: f(n(f)(x))  # λn.λf.λx.f (n f x)
PRED  = lambda n: lambda f: lambda x: n(lambda g: lambda h: h(g))(lambda u: x)(lambda u: u)  # Predecessor

**3)Define Arithmetic Operations.**

In [17]:
ADD = lambda m: lambda n: lambda f: lambda x: m(f)(n(f)(x))
MUL = lambda m: lambda n: lambda f: m(n(f))
SUB = lambda m: lambda n: n(PRED)(m)  # m - n = apply PRED n times to m

**4)Define Comparisons.**

In [18]:
IS_ZERO = lambda n: n(lambda _: FALSE)(TRUE)  # λn.n (λ_.FALSE) TRUE
GTE = lambda m: lambda n: IS_ZERO(SUB(n)(m))  # m ≥ n

**5)Evaluator for the DSL.**

In [19]:
def to_church(n):
    """Convert integer to Church numeral"""
    num = ZERO
    for _ in range(n):
        num = SUCC(num)
    return num

def church_to_int(c):
    """Convert Church numeral to integer"""
    return c(lambda x: x + 1)(0)

def church_to_bool(c):
    """Convert Church boolean to Python boolean"""
    return c(True)(False)

def evaluate(expr):
    """Evaluate DSL expressions"""
    if isinstance(expr, int):
        return to_church(expr)
    elif expr[0] == '+':
        return ADD(evaluate(expr[1]))(evaluate(expr[2]))
    elif expr[0] == '-':
        return SUB(evaluate(expr[1]))(evaluate(expr[2]))
    elif expr[0] == '*':
        return MUL(evaluate(expr[1]))(evaluate(expr[2]))
    elif expr[0] == 'if':
        return IF(evaluate(expr[1]))(evaluate(expr[2]))(evaluate(expr[3]))
    elif expr[0] == '≥':
        return GTE(evaluate(expr[1]))(evaluate(expr[2]))
    else:
        raise ValueError(f"Unknown operation: {expr[0]}")

**6)Test the DSL.**

In [20]:
if __name__ == "__main__":
    tests = [
        ("2 + 3", ['+', 2, 3], 5),
        ("3 * 4", ['*', 3, 4], 12),
        ("10 - 7", ['-', 10, 7], 3),
        ("if 0 then 5 else 6", ['if', 0, 5, 6], 6),
        ("if (1 ≥ 0) then (2*3) else (2+3)",
         ['if', ['≥', 1, 0], ['*', 2, 3], ['+', 2, 3]], 6),
        ("(5 - 2) * (3 + 1)", ['*', ['-', 5, 2], ['+', 3, 1]], 12)
    ]

    for desc, expr, expected in tests:
        result = evaluate(expr)
        # Convert result based on its type
        if isinstance(expected, int):
            output = church_to_int(result)
        else:
            output = church_to_bool(result)
        print(f"{desc:30s} → {output} (expected: {expected})")

2 + 3                          → 5 (expected: 5)
3 * 4                          → 12 (expected: 12)
10 - 7                         → <function <lambda>.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda> at 0x7df59e8b9760> (expected: 3)
if 0 then 5 else 6             → 6 (expected: 6)
if (1 ≥ 0) then (2*3) else (2+3) → 6 (expected: 6)
(5 - 2) * (3 + 1)              → <function <lambda>.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda> at 0x7df59e8b8e00> (expected: 12)
